# Stoneburner, Kurt
- ## DSC 550 - Week 09/10

The code below is mostly supplied by the book. I adapted in metrics for recall, precision and f1 scores. This assignment was only able to compile on an Intel based processor, using the following command: **pip install intel-tensorflow**. This assignment was not compatible with my home AMD based PC. It required considerable effort and frustration to work through this. Because this project needed to be built on a separate PC, I broke this assignment out from the others. Assignment 1 and 2 were hugely CPU intensive, I took advantage of letting them run on separate PCs. Hence this assignment is a bit spread out.

In [1]:
import os
import sys
# //*** Imports and Load Data
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json

#//*** Use the whole window in the IPYNB editor
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#//*** Maximize columns and rows displayed by pandas
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

### 3. Classifying Images ###
In chapter 20 of the Machine Learning with Python Cookbook, implement the code found in section 20.15 classify MSINT images using a convolutional neural network. Report the accuracy of your results.


In [1]:
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K




In [18]:
#//**************************************************************************************************************************************************************
#//*** code adpated: from https://datascience.stackexchange.com/questions/45165/how-to-get-accuracy-f1-precision-and-recall-for-a-keras-model
#//*** Functions to manually calculate recall, precision and f1 score
#//**************************************************************************************************************************************************************
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [23]:
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

# Set that the color channel value will be first
K.set_image_data_format("channels_first")
#K.set_image_data_format("channels_last")

# Set seed
np.random.seed(0)

# Set image information
channels = 1
height = 28
width = 28

# Load data and target from MNIST data
(data_train, target_train), (data_test, target_test) = mnist.load_data()

# Reshape training image data into features
data_train = data_train.reshape(data_train.shape[0], channels, height, width)

# Reshape test image data into features
data_test = data_test.reshape(data_test.shape[0], channels, height, width)

# Rescale pixel intensity to between 0 and 1
features_train = data_train / 255
features_test = data_test / 255

# One-hot encode target
target_train = np_utils.to_categorical(target_train)
target_test = np_utils.to_categorical(target_test)
number_of_classes = target_test.shape[1]

# Start neural network
network = Sequential()

# Add convolutional layer with 64 filters, a 5x5 window, and ReLU activation function
network.add(Conv2D(filters=64,
                   kernel_size=(5, 5),
                   #kernel_size=(3, 3),
                   input_shape=(channels, width, height),
                   activation='relu'))

# Add max pooling layer with a 2x2 window
network.add(MaxPooling2D(pool_size=(2, 2)))


# Add dropout layer
network.add(Dropout(0.5))

# Add layer to flatten input
network.add(Flatten())

# # Add fully connected layer of 128 units with a ReLU activation function
network.add(Dense(128, activation="relu"))

# Add dropout layer
network.add(Dropout(0.5))

# Add fully connected layer with a softmax activation function
network.add(Dense(number_of_classes, activation="softmax"))

# Compile neural network
network.compile(loss="categorical_crossentropy", # Cross-entropy
                optimizer="rmsprop", # Root Mean Square Propagation
                metrics=["accuracy",f1_m,precision_m, recall_m]
               ) # Accuracy performance metric

# Train neural network
network.fit(features_train, # Features
            target_train, # Target
            epochs=10, # Number of epochs
            verbose=1, # Don't print description after each epoch
            batch_size=1000, # Number of observations per batch
            validation_data=(features_test, target_test)) # Data for evaluation



Epoch 1/10
60/60 [==============================] - 14s 218ms/step - loss: 0.9570 - accuracy: 0.6882 - f1_m: 0.6162 - precision_m: 0.8114 - recall_m: 0.5483 - val_loss: 0.1528 - val_accuracy: 0.9564 - val_f1_m: 0.9566 - val_precision_m: 0.9673 - val_recall_m: 0.9462
Epoch 2/10
60/60 [==============================] - 13s 212ms/step - loss: 0.2002 - accuracy: 0.9408 - f1_m: 0.9405 - precision_m: 0.9560 - recall_m: 0.9255 - val_loss: 0.0914 - val_accuracy: 0.9724 - val_f1_m: 0.9726 - val_precision_m: 0.9772 - val_recall_m: 0.9681
Epoch 3/10
60/60 [==============================] - 13s 217ms/step - loss: 0.1304 - accuracy: 0.9622 - f1_m: 0.9624 - precision_m: 0.9699 - recall_m: 0.9550 - val_loss: 0.0585 - val_accuracy: 0.9813 - val_f1_m: 0.9817 - val_precision_m: 0.9848 - val_recall_m: 0.9786
Epoch 4/10
60/60 [==============================] - 14s 225ms/step - loss: 0.0985 - accuracy: 0.9720 - f1_m: 0.9719 - precision_m: 0.9775 - recall_m: 0.9664 - val_loss: 0.0546 - val_accuracy: 0.9814 

**Model Accuracy**

In [24]:
# evaluate the model
loss, accuracy, f1_score, precision, recall = network.evaluate(features_test, target_test, verbose=0)
print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')
print(f'F1 Score: {f1_score}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')

Loss: 0.03385162353515625
Accuracy: 0.9887999892234802
F1 Score: 0.9889889359474182
Precision: 0.989990234375
Recall: 0.9880191683769226
